In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/PAC

/content/drive/MyDrive/Github/PAC


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from src import data
from src.segmentation import UNet
from src import metrics, measure
from src.config import Paths
from matplotlib import rcParams
rcParams['figure.figsize'] = (3, 3)
rcParams['image.cmap'] = 'gray'

In [4]:
models = []
for filepath in Paths.models.glob('unet*'):
    if filepath.is_dir():

        logs = UNet(filepath.stem).get_logs()
        best_epoch, min_val_area_mape = logs.loc[np.argmin(logs.val_area_mape)].loc[['epoch', 'val_area_mape']]

        configs = filepath.stem.split('-')
        models.append({
            'path': filepath,
            'stem': filepath.stem,
            'loss': configs[1],
            'filters': configs[2],
            'best_epoch': best_epoch + 1,
            'min_val_area_mape': min_val_area_mape
        })
models = pd.DataFrame(models)
models

,path,stem,loss,filters,best_epoch,min_val_area_mape
0,models/unet-binary_crossentropy-16_32_64,unet-binary_crossentropy-16_32_64,binary_crossentropy,16_32_64,340.0,0.450439
1,models/unet-dice-16_32_64,unet-dice-16_32_64,dice,16_32_64,238.0,0.460099
2,models/unet-dice_topk-16_32_64,unet-dice_topk-16_32_64,dice_topk,16_32_64,146.0,0.471773
3,models/unet-dice_topk_90-16_32_64,unet-dice_topk_90-16_32_64,dice_topk_90,16_32_64,102.0,0.853911
4,models/unet-dice-4_8_16_32,unet-dice-4_8_16_32,dice,4_8_16_32,794.0,0.504615
5,models/unet-dice_topk_30-32_64,unet-dice_topk_30-32_64,dice_topk_30,32_64,1.0,145.849396
6,models/unet-binary_crossentropy-16_32_64-ii,unet-binary_crossentropy-16_32_64-ii,binary_crossentropy,16_32_64,480.0,0.418916


In [5]:
def loss_by_name(name):
    loss = None
    try:
        loss = getattr(metrics, name)
    except:
        try: loss = getattr(tf.keras.losses, name)
        except: raise Exception(f'Função {name} não encontrada em tf.keras.losses ou em src.metrics')
    return loss

def get_metrics(model_name):
    logs = UNet(model_name).get_logs()
    my_metrics = []
    for col in logs.columns:
        if col not in ('epoch', 'loss') and 'val' not in col:
            try: my_metrics.append(getattr(metrics, col))
            except:
                try: my_metrics.append(getattr(tf.keras.metrics, col))
                except: raise Exception(f'Função {col} não encontrada em tf.keras.losses ou em src.metrics')
    return my_metrics

In [6]:
dataset = (x_train, y_train), (x_test, y_test) = data.load_dataset(augmentation=True, grayscale=True, norm=True)

In [7]:
best_model_info = models.loc[np.argmin(models.min_val_area_mape)]
best_model = UNet(best_model_info.stem, dataset).load(compile=False)
best_model.load_weights(int(best_model_info.best_epoch))
best_model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss= loss_by_name(best_model_info.loss),
    metrics= get_metrics(best_model_info.stem)
)

In [8]:
best_model.evaluate()

2/2 [==============================] - 12s 4s/step - loss: 0.0169 - DSC: 0.9957 - IoU: 0.9914 - area_mape: 0.4171


[0.0169413760304451, 0.995664656162262, 0.9913729429244995, 0.417097806930542]

In [10]:
best_model_info.best_epoch

480.0